In [ ]:
# import modules
import pandas as pd
import geopandas as gpd
import rasterstats
import rasterio
from shapely.geometry import Polygon, LineString, Point 
import numpy as np 
import tqdm
from tqdm.notebook import tqdm_notebook
import time
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning) 
pd.options.mode.chained_assignment = None

In [ ]:
### STEP 1 - Find nearest wave point for each transect ####
# read transect points to gpd and tidy up
#set engine to pyogrio
engine='pyogrio'
transects = gpd.read_file('data/Intersect_v2.shp', engine=engine).to_crs(2193)

# list of attributes to keep
to_keep = ['BaselineID', 'TransectID', 'NSM', 'EPR', 'WLR', 'WCI90', 'DSASdate', 'geometry']
# iterate over cols and drop unwanted cols
to_drop = []
for col in transects.columns:
    if col not in to_keep:
        to_drop.append(col)
print(to_drop)
transects.drop(to_drop, axis=1, inplace=True)

print(transects.value_counts('TransectID'))
transects.head()


In [ ]:
# read wave data incl CD as gpd dataframe
wave_points_df = pd.read_csv('data/wave_points_nztm.csv')
wave_points_gdf = gpd.GeoDataFrame(wave_points_df, geometry=gpd.points_from_xy(wave_points_df.X, wave_points_df.Y), crs=2193)
wave_points_gdf.rename(columns={'X': 'wave_X', 'Y': 'wave_Y', 'ID': 'waveID'}, inplace=True)
wave_points_gdf.head()

# Perform nearest sjoin to find nearest wave_point for each transect
transect_wp = gpd.sjoin_nearest(transects, wave_points_gdf, how='inner')
transect_wp.head()

### END OF STEP 1 ###

In [ ]:
#### STEP 2 - Create .shp for transects from intersection to wave point ####
# create new gdf where geometry is Linestring from transect intersect to wave point
# function to generate Linestring geometry
def get_transect_geom(row):
    eov_x = row.geometry.x
    eov_y = row.geometry.y
    wave_x = row.wave_X
    wave_y = row.wave_Y
    return  LineString([(eov_x, eov_y), (wave_x, wave_y)])

def get_eov_x(row):
    eov_x = row.geometry.x
    return  eov_x

def get_eov_y(row):
    eov_y = row.geometry.y
    return  eov_y

# create new column containing Linestring from transect to wave point
transect_wp['transect_wp_geom'] = transect_wp.apply(lambda row : get_transect_geom(row), axis=1)
print(transect_wp.head())

# store intersect 
transect_wp['intsect_X'] = transect_wp.apply(lambda row : get_eov_x(row), axis=1)
transect_wp['intsect_Y'] = transect_wp.apply(lambda row : get_eov_y(row), axis=1)

# set geometry to transect_wp Linestring
# tidy up gdf and set geometry to transect
transect_wp.set_geometry('transect_wp_geom', inplace=True, crs=2193)
#transect_wp.drop(['index_right'], axis=1, inplace=True)

transect_wp.drop(columns=['geometry'], axis=1, inplace=True)
transect_wp.head()

### END OF STEP 2 ###

In [ ]:
### STEP 3 FUNCTIONS ### 
# function to find depth corresponding to closure depth
def get_depth(df):
    # define cd
    cd = df.CD
    
    # return the bathy_dpth value closest to CD 
    bthy_dpth = df.iloc[(df['bathy_dpth']--abs(cd.iloc[0])).abs().argsort()[:1]] # using -abs() to ensure cd depth values match bathy_dpth
    
    return bthy_dpth

# function to find length between
def get_length(df):
    # create gdf to find length of transect from elevation to depth
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.intsect_X, df.intsect_Y), crs=2193)
   
    # redefine geometry from bathy_depth to eov_point and set as geometry
    gdf['transect'] = LineString([(df.intsect_X, df.intsect_Y), (df.coord_X, df.coord_Y)])
    gdf.set_geometry('transect', inplace=True, crs=2193)
    
    # calc length of transect
    gdf['length'] = gdf.length
    
    return gdf




In [ ]:
#### STEP 3 - FIND DEPTH VALUES ####

### STEP 3 FUNCTIONS ### 
# function to find depth corresponding to closure depth
def get_depth(df):
    # define cd
    cd = df.CD
    
    # return the bathy_dpth value closest to CD 
    bthy_dpth = df.iloc[(df['bathy_dpth']--abs(cd.iloc[0])).abs().argsort()[:1]] # using -abs() to ensure cd depth values match bathy_dpth
    
    return bthy_dpth

# function to find length between
def get_length(df):
    # create gdf to find length of transect from elevation to depth
    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.intsect_X, df.intsect_Y), crs=2193)
   
    # redefine geometry from bathy_depth to eov_point and set as geometry
    gdf['transect'] = LineString([(df.intsect_X, df.intsect_Y), (df.coord_X, df.coord_Y)])
    gdf.set_geometry('transect', inplace=True, crs=2193)
    
    # calc length of transect
    gdf['length'] = gdf.length
    
    return gdf

In [ ]:
# FIND BATHY VALUES WITHIN 250m OF EACH TRANSECT
# read bathymetry centroids to gdf
start = time.time()
bathy = gpd.read_file('data/Bathy250m_centroids_nztm.shp', engine=engine).to_crs(2193)
end = time.time()
print('time to read bathmetry: ', end - start, 'seconds')

start = time.time()
# get coordinates for bathy depth for next step
bathy['coord_X'] = bathy.apply(lambda row : get_eov_x(row), axis=1)
bathy['coord_Y'] = bathy.apply(lambda row : get_eov_y(row), axis=1)

# generate buffers for each transect and set_geometry to buffer polygons
transect_wp['buffer'] = transect_wp.buffer(250)
transect_wp.set_geometry('buffer', inplace=True, crs=2193)
# transect_wp.drop(['index_right'], axis=1, inplace=True)
transect_wp.head()

# spatial join to find all bathy_points that lie within transect buffers 
points_within = gpd.sjoin(bathy, transect_wp,  how='inner', predicate='within')
# drop join columns
points_within.drop(['index_right'], axis=1, inplace=True)

end = time.time()
print('time to perform spatial join: ', end - start, 'seconds')
points_within.head()


### RETURN BATHY DEPTH and length of each transect 
# iterate over each transect and find the bathy_dpth closest to cd and eov elevation
# create empty list of df with same columns
df_list = []
# get list of transect IDs
id_list = points_within.TransectID.unique().tolist()

# gen start time
start = time.time()

for i in tqdm_notebook(id_list, desc='Finding bathymetry depth: '):
    # return rows corresponding to a given transect
    rows = points_within.query('TransectID == @i')

    # get bathymetry depth
    depth = get_depth(rows)

    # get length 
    gdf = get_length(depth)

    # append to dataframe list
    df_list.append(gdf)

end = time.time()
elapsed = end - start
print('run time: ', elapsed, 'seconds')

# concat dataframe list and write to csv file
df_bthy_dpth = pd.DataFrame(pd.concat(df_list))  

# save to csv
df_bthy_dpth.to_csv('all_depth_length.csv')
df_bthy_dpth.head()

In [ ]:
### STEP 4 - CALC ELEVATION BASED ON EOV POINT

## FUNCTIONS ##
# function to find EOV elevation from LIDAR
def get_elevation(df_row, raster):
    # extract xy from columns
    x = df_row.intsect_X
    y = df_row.intsect_Y
    # create point from x, y
    point = Point(x,y)
    # point query from rasterstats returning elevation value
    ele_val = rasterstats.point_query(point, raster)
    print(df_row.TransectID)
    print("Raster value on point %.2f \n"%ele_val[0])
    
    return ele_val[0]

## END OF FUNCTIONS ##

# read in and join LIDAR tile ids
tile_id = gpd.read_file('/path/to/lidar/tiles/shapefile').to_crs(2193)

# spatial join to get tile_id
# spatial join to find all bathy_points that lie within transect buffers 
elevation_points = gpd.sjoin(df_bthy_dpth, tile_id,  how='inner', predicate='within')

elevation_points.drop(['field_1','index_right', 'topo50_cov'], axis=1, inplace=True)

print(elevation_points.value_counts('sheet_code'))

elevation_points.head()



In [ ]:
# create list of tile_id to iterate over
tile_id_list = elevation_points.sheet_code.unique().tolist()

# create blank list to save df to
df_list = []

start = time.time()
for i in  tqdm_notebook(tile_id_list,desc='Finding elevation: '):  
    rows = elevation_points.query('sheet_code == @i')
    # read LUT containing LiDAR tile file paths
    tile_lut = pd.read_csv('data/LIDARTILES.csv')
    # find tile using LUT of tile file paths
    tile_id = rows.sheet_code
    #print('finding tile: ' + tile_id)
    lidar_fp = ''
    for i in range(len(tile_lut)):
           if tile_lut.loc[i, "TILES"] == tile_id.iloc[0]:
                 lidar_fp += tile_lut.loc[i, "FP"]                
    print('opening LiDAR tile: ' + lidar_fp)
    rows['elevation'] = rows.apply(lambda row: get_elevation(row, lidar_fp), axis=1)
    df_list.append(rows)

end = time.time()
print('time to find elevation values from lidar: ', end - start, 'seconds') 


# # concat dataframe list and write to csv df_list))
df_nearly_there = pd.DataFrame(pd.concat(df_list))
df_nearly_there.to_csv('all_CEHZ_variables.csv')
df_nearly_there.head()